# Set Up

In [25]:
import pandas as pd

In [26]:
index_torgo = pd.read_csv('index_TORGO.txt', sep = '|')
index_torgo_filtered = index_torgo[(~index_torgo['prompt'].str.contains(']'))
                                   & (index_torgo['speaker'] == 'F01')
                                   & (~index_torgo['prompt'].str.contains('/'))]
index_torgo_filtered['url'] =   'https://s3.amazonaws.com/capstone-voice-for-all/TORGO/' \
                                + index_torgo_filtered['speaker'] + '/' \
                                + index_torgo_filtered['session'] + '/' \
                                + index_torgo_filtered['mic'] + '/' \
                                + index_torgo_filtered['prompt_id'] + '.wav'
index_torgo_filtered['filepath'] = '/home/jovyan/work/data/TORGO/' \
                                + index_torgo_filtered['speaker'] + '/' \
                                + index_torgo_filtered['session'] + '/' \
                                + index_torgo_filtered['mic'] + '/' \
                                + index_torgo_filtered['prompt_id'] + '.wav'

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [40]:
pd.options.display.max_colwidth = 200
# index_torgo_filtered.sample(10)
index_torgo_filtered[['prompt', 'url', 'filepath']].head(10)
index_torgo_filtered_top10 = index_torgo_filtered[['prompt', 'url', 'filepath']].sample(10)
index_torgo_filtered_top10

,prompt,url,filepath
9,up,https://s3.amazonaws.com/capstone-voice-for-all/TORGO/F01/Session1/wav_headMic/0010.wav,/home/jovyan/work/data/TORGO/F01/Session1/wav_headMic/0010.wav
16,double,https://s3.amazonaws.com/capstone-voice-for-all/TORGO/F01/Session1/wav_headMic/0017.wav,/home/jovyan/work/data/TORGO/F01/Session1/wav_headMic/0017.wav
94,glow,https://s3.amazonaws.com/capstone-voice-for-all/TORGO/F01/Session1/wav_headMic/0095.wav,/home/jovyan/work/data/TORGO/F01/Session1/wav_headMic/0095.wav
74,warm,https://s3.amazonaws.com/capstone-voice-for-all/TORGO/F01/Session1/wav_headMic/0075.wav,/home/jovyan/work/data/TORGO/F01/Session1/wav_headMic/0075.wav
215,rate,https://s3.amazonaws.com/capstone-voice-for-all/TORGO/F01/Session1/wav_arrayMic/0082.wav,/home/jovyan/work/data/TORGO/F01/Session1/wav_arrayMic/0082.wav
57,the,https://s3.amazonaws.com/capstone-voice-for-all/TORGO/F01/Session1/wav_headMic/0058.wav,/home/jovyan/work/data/TORGO/F01/Session1/wav_headMic/0058.wav
153,Usually minus several buttons.,https://s3.amazonaws.com/capstone-voice-for-all/TORGO/F01/Session1/wav_arrayMic/0020.wav,/home/jovyan/work/data/TORGO/F01/Session1/wav_arrayMic/0020.wav
251,knew,https://s3.amazonaws.com/capstone-voice-for-all/TORGO/F01/Session1/wav_arrayMic/0118.wav,/home/jovyan/work/data/TORGO/F01/Session1/wav_arrayMic/0118.wav
226,"A long, flowing beard clings to his chin,",https://s3.amazonaws.com/capstone-voice-for-all/TORGO/F01/Session1/wav_arrayMic/0093.wav,/home/jovyan/work/data/TORGO/F01/Session1/wav_arrayMic/0093.wav
100,bug,https://s3.amazonaws.com/capstone-voice-for-all/TORGO/F01/Session1/wav_headMic/0101.wav,/home/jovyan/work/data/TORGO/F01/Session1/wav_headMic/0101.wav


## Amazon Transcribe

[Info about Amazon Transcribe](https://aws.amazon.com/transcribe/)

In [28]:
from __future__ import print_function
from urllib.request import urlopen
import time
import boto3
import json

In [29]:
def start_transcription_job(row):
    jobName = str(row.name)
    jobUri = row.url
    transcribe.start_transcription_job(
        TranscriptionJobName=jobName,
        Media={'MediaFileUri': jobUri},
        MediaFormat='wav',
        LanguageCode='en-US'
    )

In [30]:
def get_transcription_job(row):
    jobName = str(row.name)
    status = transcribe.get_transcription_job(TranscriptionJobName=jobName)
    if status['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED', 'FAILED']:
        result_url = status['TranscriptionJob']['Transcript']['TranscriptFileUri']
        with urlopen(result_url) as conn:
            transcript_html = conn.read()
            transcript = json.loads(transcript_html)
            print(jobName + ': ' + transcript['results']['transcripts'][0]['transcript'])
    else:
        print(jobName + ' not done yet')

In [31]:
def delete_transcription_job(row):
    jobName = str(row.name)
    transcribe.delete_transcription_job(TranscriptionJobName=jobName)

In [134]:
# index_torgo_filtered_top10.apply(start_transcription_job, axis=1)
index_torgo_filtered_top10.apply(get_transcription_job, axis=1)
# index_torgo_filtered_top10.apply(delete_transcription_job, axis=1)

80: Sue.
190: 
34: No, No. Who moved him? Kill them? No. Yeah, mostly.
90: Thank you.
174: Play.
35: We have off Teo once more and smokeless.
232: Good morning.
142: Huh?
264: Cool.
117: New.


80     None
190    None
34     None
90     None
174    None
35     None
232    None
142    None
264    None
117    None
dtype: object

## Google Cloud Speech-to-Text

[Info about Speech-to-Text](https://cloud.google.com/speech-to-text/)

In [1]:
import io
import os

# Imports the Google Cloud client library
from google.cloud import speech
from google.cloud.speech import enums
from google.cloud.speech import types

In [16]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/home/jovyan/work/capstone-c60aec7ead62.json"

In [43]:
def google_translate(row):
    jobName = str(row.name)
    file_name = row.filepath
    with io.open(file_name, 'rb') as audio_file:
        content = audio_file.read()
        audio = types.RecognitionAudio(content=content)

    config = types.RecognitionConfig(
        encoding=enums.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=16000,
        language_code='en-US')

    # Detects speech in the audio file
    response = client.recognize(config, audio)
    try:
        transcript = response.results[0].alternatives[0].transcript
        print(jobName + ': ' + transcript)
    except:
        print(jobName + ' has no results.')

In [44]:
index_torgo_filtered_top10

,prompt,url,filepath
9,up,https://s3.amazonaws.com/capstone-voice-for-all/TORGO/F01/Session1/wav_headMic/0010.wav,/home/jovyan/work/data/TORGO/F01/Session1/wav_headMic/0010.wav
16,double,https://s3.amazonaws.com/capstone-voice-for-all/TORGO/F01/Session1/wav_headMic/0017.wav,/home/jovyan/work/data/TORGO/F01/Session1/wav_headMic/0017.wav
94,glow,https://s3.amazonaws.com/capstone-voice-for-all/TORGO/F01/Session1/wav_headMic/0095.wav,/home/jovyan/work/data/TORGO/F01/Session1/wav_headMic/0095.wav
74,warm,https://s3.amazonaws.com/capstone-voice-for-all/TORGO/F01/Session1/wav_headMic/0075.wav,/home/jovyan/work/data/TORGO/F01/Session1/wav_headMic/0075.wav
215,rate,https://s3.amazonaws.com/capstone-voice-for-all/TORGO/F01/Session1/wav_arrayMic/0082.wav,/home/jovyan/work/data/TORGO/F01/Session1/wav_arrayMic/0082.wav
57,the,https://s3.amazonaws.com/capstone-voice-for-all/TORGO/F01/Session1/wav_headMic/0058.wav,/home/jovyan/work/data/TORGO/F01/Session1/wav_headMic/0058.wav
153,Usually minus several buttons.,https://s3.amazonaws.com/capstone-voice-for-all/TORGO/F01/Session1/wav_arrayMic/0020.wav,/home/jovyan/work/data/TORGO/F01/Session1/wav_arrayMic/0020.wav
251,knew,https://s3.amazonaws.com/capstone-voice-for-all/TORGO/F01/Session1/wav_arrayMic/0118.wav,/home/jovyan/work/data/TORGO/F01/Session1/wav_arrayMic/0118.wav
226,"A long, flowing beard clings to his chin,",https://s3.amazonaws.com/capstone-voice-for-all/TORGO/F01/Session1/wav_arrayMic/0093.wav,/home/jovyan/work/data/TORGO/F01/Session1/wav_arrayMic/0093.wav
100,bug,https://s3.amazonaws.com/capstone-voice-for-all/TORGO/F01/Session1/wav_headMic/0101.wav,/home/jovyan/work/data/TORGO/F01/Session1/wav_headMic/0101.wav


In [45]:
client = speech.SpeechClient()

index_torgo_filtered_top10.apply(google_translate, axis=1)

9 has no results.
16: gobble
94: grow
74: warm
215: all right
57 has no results.
153: why you so buttons
251: new
226: bowling in Queens
100: mug


9      None
16     None
94     None
74     None
215    None
57     None
153    None
251    None
226    None
100    None
dtype: object